In [ ]:
#Basics
import os
import io
import re
import string
from tqdm import tqdm
import numpy as np

#tf libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

#torch libraries
import torch
from torch import cuda

#Time related libraries
import datetime
from timeit import default_timer as timer

#General libraries
import collections
import math
import random
import matplotlib as mpl
import matplotlib.pyplot as plt

#Dealing with files, data and general stuff
import zipfile
import pandas as pd

#May be used but very unlikely.
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
from numba import jit, cuda #Former way of GPU acceleration in my project.

#Transformers Library. Taken from "Hugging Face".
from transformers import BertConfig, BertModel, BertTokenizer, BertForMaskedLM, BertTokenizerFast, pipeline
from tokenizers import BertWordPieceTokenizer
from tokenizers.processors import BertProcessing

#Optimizers taken from the "Transformers" Library that I have mentioned above as well.
from transformers import AdamW

In [ ]:
#Loading The Tokenizer & Model Path
path_to_tokenizer = "G:/VisualStudioCodeG/Resources/Tokenizers/bookcorpus/Run1/BERTbookcorpusTokenizerRun1-vocab.txt"
path_to_model = "G:/VisualStudioCodeG/Resources/Models/bookcorpus/Run1"

In [ ]:
tokenizer = BertTokenizer.from_pretrained(path_to_tokenizer, clean_text=True, max_length=512)

c:\Users\Cthulhu\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:1706: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [ ]:
model = BertForMaskedLM.from_pretrained(path_to_model)
#model.eval()

In [ ]:
pipe = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [ ]:
ExampleSentence = "How [MASK] you doing?"

In [ ]:
inputs = tokenizer(ExampleSentence, return_tensors="pt") #Tokenizing the sentence after modifications.

with torch.no_grad():
    logits = model(**inputs).logits
        #Using the Model to get the "raw prediction vector" which will be passed to the softmax function when predicting the missing word.

mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0] #Retrieving the index of the [MASK]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
PredWord = tokenizer.decode(predicted_token_id)

print(PredWord)

are


In [ ]:
pipe(f'How {pipe.tokenizer.mask_token} you doing?')

[{'score': 0.21366684138774872,
  'token': 309,
  'token_str': 'are',
  'sequence': 'how are you doing?'},
 {'score': 0.14240136742591858,
  'token': 215,
  'token_str': 'is',
  'sequence': 'how is you doing?'},
 {'score': 0.09299653768539429,
  'token': 287,
  'token_str': 'about',
  'sequence': 'how about you doing?'},
 {'score': 0.07042302936315536,
  'token': 179,
  'token_str': 'do',
  'sequence': 'how do you doing?'},
 {'score': 0.04870709031820297,
  'token': 142,
  'token_str': 'was',
  'sequence': 'how was you doing?'}]